# 06_Base_sumarizada

## 01_import_bibliotecas

In [1]:
# === Geração da base consolidada para análise regional por estado + cluster ===

import pandas as pd
import numpy as np
from datetime import datetime
import joblib
from io import BytesIO
import requests
from sklearn.preprocessing import StandardScaler

## 02_Carga_de_dados

In [2]:
candidatos_sum = 'https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/fontes_tratadas/04_dados_candidatos.parquet'

In [3]:
df_2 = pd.read_parquet(candidatos_sum)

## 03_Tratamento

In [4]:
# === 2. Calcular idade com base em data_nascimento ===
hoje = pd.to_datetime("today")
df_2["data_nascimento"] = pd.to_datetime(df_2["data_nascimento"], errors="coerce")
df_2["idade"] = df_2["data_nascimento"].apply(lambda x: hoje.year - x.year if pd.notnull(x) else np.nan)


C:\Users\pedro\AppData\Local\Temp\ipykernel_11556\438117030.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_2["data_nascimento"] = pd.to_datetime(df_2["data_nascimento"], errors="coerce")


In [5]:
# 2.2. Mapear nomes de estado para siglas
mapa_estados = {
    "acre": "AC", "alagoas": "AL", "amapá": "AP", "amazonas": "AM", "bahia": "BA",
    "ceará": "CE", "distrito federal": "DF", "espírito santo": "ES", "goiás": "GO",
    "maranhão": "MA", "mato grosso": "MT", "mato grosso do sul": "MS", "minas gerais": "MG",
    "pará": "PA", "paraíba": "PB", "paraná": "PR", "pernambuco": "PE", "piauí": "PI",
    "rio de janeiro": "RJ", "rio grande do norte": "RN", "rio grande do sul": "RS",
    "rondônia": "RO", "roraima": "RR", "santa catarina": "SC", "são paulo": "SP",
    "sergipe": "SE", "tocantins": "TO"
}
df_2["estado"] = df_2["endereco"].str.lower().map(mapa_estados)

In [6]:
# === 3. Carregar modelo e colunas usadas ===

modelo = joblib.load(BytesIO(requests.get("https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/03_modelos/modelo_kmeans.joblib").content))

colunas_modelo = joblib.load(BytesIO(requests.get(f"https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/03_modelos/colunas_usadas.joblib").content))

In [7]:
# Tratamento de nivel_educacional
mapa_educacional = {
    'ensino fundamental': 1,
    'ensino médio': 2,
    'ensino superior': 2,
    'superior incompleto': 2,
    'pós-graduação ou mais': 3,
    'não identificado': 0,
    None: 0
}
df_2['nivel_educacional'] = df_2['nivel_educacional'].map(mapa_educacional).fillna(0).astype(int)

In [8]:
# Conversão de variável booleana experiencia_sap
df_2['experiencia_sap'] = df_2['experiencia_sap'].astype(int)

In [9]:
# One-hot encoding para variáveis categóricas
colunas_categoricas = ['categoria_profissional', 'subarea_profissional', 'nivel_hierarquico']
df_2 = pd.get_dummies(df_2, columns=colunas_categoricas, drop_first=True)

In [10]:
# Escalonamento de variáveis contínuas
variaveis_continuas = ['remuneracao_zscore', 'tempo_experiencia_anos', 'quantidade_experiencias']
scaler = StandardScaler()
df_2[variaveis_continuas] = scaler.fit_transform(df_2[variaveis_continuas])

In [11]:
# Tratar valores ausentes
colunas_com_nan = df_2.columns[df_2.isnull().any()].tolist()
for col in colunas_com_nan:
    df_2[f"tem_{col}"] = df_2[col].notnull().astype(int)
    df_2[col] = df_2[col].fillna(-9999)

### Entrada_do_modelo

In [12]:
df_2

,id_candidato,data_nascimento,remuneracao_mensal_brl,remuneracao_zscore,tempo_experiencia_anos,quantidade_experiencias,nivel_educacional,experiencia_sap,endereco,sexo,...,nivel_hierarquico_Pleno,nivel_hierarquico_Sênior,nivel_hierarquico_Trainee,nivel_hierarquico_Técnico,tem_data_nascimento,tem_remuneracao_mensal_brl,tem_remuneracao_zscore,tem_tempo_experiencia_anos,tem_idade,tem_estado
0,31000,-9999,-9999.0,-9999.000000,-0.535771,1.598060,2,1,,,...,0,0,0,0,0,0,0,1,0,0
1,31001,1994-12-28 00:00:00,1900.0,-0.048531,-0.535771,-0.424810,2,0,são paulo,Feminino,...,0,0,0,0,1,1,1,1,1,1
2,31002,1988-12-12 00:00:00,3795.0,-0.028022,-0.158345,0.839484,2,1,são paulo,Feminino,...,0,0,0,0,1,1,1,1,1,1
3,31003,1992-08-05 00:00:00,1100.0,-0.057189,-0.082860,1.092342,2,0,são paulo,Feminino,...,0,0,0,0,1,1,1,1,1,1
4,31004,1969-12-31 00:00:00,-9999.0,-9999.000000,0.294566,0.080907,2,0,,,...,0,0,0,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,5995,-9999,0.0,-0.069094,-9999.000000,-0.424810,0,0,,,...,0,0,0,0,0,1,1,0,0,0
42478,5996,-9999,0.0,-0.069094,-9999.000000,-0.424810,0,0,,,...,1,0,0,0,0,1,1,0,0,0
42479,5997,-9999,0.0,-0.069094,-9999.000000,-0.424810,0,0,,,...,1,0,0,0,0,1,1,0,0,0
42480,5998,-9999,0.0,-0.069094,-9999.000000,-0.424810,0,0,,,...,1,0,0,0,0,1,1,0,0,0


In [13]:
# Remover colunas não utilizadas
colunas_remover = [
    'data_nascimento',
    'id_candidato',
    'remuneracao_mensal_brl',
    'endereco',
    'estado',
]

df_model = df_2.drop(columns=colunas_remover, errors='ignore')

In [14]:
# === 4. Preparar features de entrada ===
# df_model = df_2.copy()

# Preencher flags e sentinelas
for col in colunas_modelo:
    if col.startswith("tem_"):
        original = col.replace("tem_", "")
        df_model[col] = df_model[original].notnull().astype(int)
        df_model[original] = df_model[original].fillna(-9999)

# Garantir todas as colunas
for col in colunas_modelo:
    if col not in df_model.columns:
        df_model[col] = 0

In [15]:
# === 5. Prever cluster para cada candidato ===
X_input = df_model[colunas_modelo]
df_2["cluster"] = modelo.predict(X_input)

In [16]:
# 2.10. Faixa etária
bins = [0, 24, 34, 44, 54, 64, 120]
labels = ["até 24", "25 a 34", "35 a 44", "45 a 54", "55 a 64", "65+"]
df_2["faixa_etaria"] = pd.cut(df_2["idade"], bins=bins, labels=labels, right=True)

In [17]:
df_2.head()

,id_candidato,data_nascimento,remuneracao_mensal_brl,remuneracao_zscore,tempo_experiencia_anos,quantidade_experiencias,nivel_educacional,experiencia_sap,endereco,sexo,...,nivel_hierarquico_Trainee,nivel_hierarquico_Técnico,tem_data_nascimento,tem_remuneracao_mensal_brl,tem_remuneracao_zscore,tem_tempo_experiencia_anos,tem_idade,tem_estado,cluster,faixa_etaria
0,31000,-9999,-9999.0,-9999.000000,-0.535771,1.598060,2,1,,,...,0,0,0,0,0,1,0,0,0,NaN
1,31001,1994-12-28 00:00:00,1900.0,-0.048531,-0.535771,-0.424810,2,0,são paulo,Feminino,...,0,0,1,1,1,1,1,1,2,25 a 34
2,31002,1988-12-12 00:00:00,3795.0,-0.028022,-0.158345,0.839484,2,1,são paulo,Feminino,...,0,0,1,1,1,1,1,1,2,35 a 44
3,31003,1992-08-05 00:00:00,1100.0,-0.057189,-0.082860,1.092342,2,0,são paulo,Feminino,...,0,0,1,1,1,1,1,1,2,25 a 34
4,31004,1969-12-31 00:00:00,-9999.0,-9999.000000,0.294566,0.080907,2,0,,,...,0,0,1,0,0,1,1,0,0,55 a 64


In [18]:
# === 6. Agregar por estado + cluster ===
df_sumarizado = df_2.groupby(["estado", "cluster", "faixa_etaria"]).agg(
    candidatos=("id_candidato", "count"),
    escolaridade_media=("nivel_educacional", "mean"),
    perc_com_sap=("experiencia_sap", "mean"),
    remuneracao_media=("remuneracao_mensal_brl", lambda x: x.replace(-9999, np.nan).mean()),
    experiencias_media=("quantidade_experiencias", "mean")
).reset_index()

# Arredondamentos
df_sumarizado["escolaridade_media"] = df_sumarizado["escolaridade_media"].round(2)
df_sumarizado["perc_com_sap"] = (df_sumarizado["perc_com_sap"] * 100).round(1)
df_sumarizado["remuneracao_media"] = df_sumarizado["remuneracao_media"].round(2)
df_sumarizado["experiencias_media"] = df_sumarizado["experiencias_media"].round(2)

## 04_Salvando_base

In [19]:
df_sumarizado.head(25)

,estado,cluster,faixa_etaria,candidatos,escolaridade_media,perc_com_sap,remuneracao_media,experiencias_media
0,-9999,0,até 24,1,3.00,100.0,NaN,0.08
1,-9999,0,25 a 34,5,0.80,0.0,NaN,-0.02
2,-9999,0,35 a 44,8,1.62,25.0,NaN,0.49
3,-9999,0,45 a 54,8,1.75,12.5,NaN,0.43
4,-9999,0,55 a 64,1221,1.49,38.4,NaN,0.64
5,-9999,0,65+,0,NaN,NaN,NaN,NaN
6,-9999,1,até 24,2,1.00,0.0,1165.00,-0.42
7,-9999,1,25 a 34,138,0.09,0.0,1577.90,-0.42
8,-9999,1,35 a 44,168,0.06,0.0,1742.21,-0.42
9,-9999,1,45 a 54,74,0.18,2.7,2607.36,-0.42


In [20]:
Caminho = "C:\\Users\\pedro\\Documents\\Área de Trabalho\\tech_challenge_5\\01_fontes\\arquivos_decision\\fontes_tratadas\\"

In [21]:
df_sumarizado["estado"] = df_sumarizado["estado"].astype(str)

In [22]:
# === 7. Exportar para Parquet e Excel ===
df_sumarizado.to_parquet(Caminho + "05_dados_sumarizados.parquet", index=False)

df_sumarizado.to_excel(Caminho + "05_dados_sumarizados.xlsx", index=False)